In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.8


In [2]:
from classifier.create_ws import AZHelper
ws = AZHelper.load_ws()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/aaditya/PycharmProjects/Codefundopp/classifier/aml_config/config.json


Workspace name: pyt1
Azure region: eastus2
Subscription id: af7772d6-3275-4058-8d9e-7640742004e3
Resource group: Codefundopp


In [4]:
compute_target = AZHelper.load_cp(ws)

creating a new compute target...


Creating


Succeeded

AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-01-21T09:43:50.888000+00:00', 'creationTime': '2019-01-21T09:40:14.475546+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-01-21T09:43:56.853257+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
import os

project_folder = './pytorch-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

In [67]:
import shutil

shutil.copy('classifier/train_on_azure.py', project_folder)
shutil.copy('classifier/dataset.py', project_folder)
shutil.copy('classifier/xception.py', project_folder)

'./pytorch-distr-hvd/xception.py'

In [8]:
from azureml.core import Experiment

experiment_name = 'pytorch-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

In [9]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob pyt1storagekitusvbf azureml-blobstore-7108e7e0-f9a2-45aa-99f1-27e69744c740


In [ ]:
ds.upload(src_dir='data_wf', target_path='data_wf', overwrite=True, show_progress=False)


In [52]:
ds.upload(src_dir='classifier/weights', target_path='weights', overwrite=True, show_progress=True)


Uploading classifier/weights/xception_imagenet.pth


Uploaded classifier/weights/xception_imagenet.pth, 1 files out of an estimated total of 1


$AZUREML_DATAREFERENCE_182b99fdd3a743d2950a8c12c84befcc

In [65]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data-folder': ds.as_mount(),
    '--epochs': 30,
    '--batch-size': 8,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='train_on_azure.py',
                    script_params=script_params,
                    node_count=4,
                    process_count_per_node=1,
                    distributed_backend='mpi',
                    use_gpu=True)

In [66]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: pytorch-distr-hvd,
Id: pytorch-distr-hvd_1548074667178,
Type: azureml.scriptrun,
Status: Queued)


In [14]:
from azureml.widgets import RunDetails

RunDetails(run).show()

ModuleNotFoundError: No module named 'azureml.widgets'

In [51]:
print(run)

Run(Experiment: pytorch-distr-hvd,
Id: pytorch-distr-hvd_1548069190034,
Type: azureml.scriptrun,
Status: Running)
